In [1]:
import numpy as np
import torch
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [15]:
class CustomDataset(Dataset):
    def __init__(self, analyze_ticks, result_ticks, max_ticks):
        self.data_path = "testingData.csv"
        self.data = []
        df = pd.read_csv(self.data_path, encoding='utf-16', delimiter=";", header=None)
        ticksArray = np.array(df.values)
        if max_ticks>ticksArray.shape[0]:
            max_ticks=ticksArray.shape[0]
        # max_sum = 0;
        for idx in np.arange(analyze_ticks-1, max_ticks-result_ticks-1):
            
            analyze_set = np.array([])
            for i in np.arange(analyze_ticks-1, 0, -1):
                analyze_set = np.append(analyze_set, ticksArray[idx-i], 0)
            result_sum = 0
            for i in np.arange(result_ticks):
                result_sum+=ticksArray[idx+i+1][1]
            # if abs(result_sum)>max_sum:
            #     max_sum=abs(result_sum)
            
            if(abs(result_sum)>20):
                self.data.append([analyze_set, result_sum])
            elif np.random.rand()<0.1:
                self.data.append([analyze_set, result_sum])
            # self.data.append([analyze_set, result_sum])
            
        # print(max_sum)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        analyze_set, result_sum = self.data[idx]
        analyze_tensor = torch.from_numpy(analyze_set).to(torch.float32)
        result_tensor = torch.tensor([result_sum]).to(torch.float32)
        return analyze_tensor, result_tensor

In [16]:
dataset = CustomDataset(51,20,60000)

batch_size0 = 10

train_dataloader = DataLoader(dataset, batch_size=batch_size0, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=batch_size0, shuffle=True)


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
        )
        # print(self.linear_relu_stack[0].weight.dtype)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [279]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)


In [11]:
learning_rate = 1e-3


In [8]:
def train_loop(dataloader, model, loss_fn, optimizer, log):
    size = len(dataloader.dataset)
    one_time = True
    for batch, (X, y) in enumerate(dataloader):
        
        # Compute prediction and loss
        pred = model(X)
        if one_time:
            one_time= False
            # print(pred)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0 and log:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, accuracy, log):
    # size = len(dataloader.dataset)
    size = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            for idx in range(pred.shape[0]):
                if abs(pred[idx][0])>20:
                    size +=1
                    if(pred[idx][0]+accuracy>=y[idx][0] and pred[idx][0]-accuracy<=y[idx][0]):
                        correct += 1

    test_loss /= num_batches
    correct /= size
    if log:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:


epochs = 500
for t in range(epochs):
    log = False
    if (t+1)%50==0:
        log = True
        print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, log)
    test_loop(test_dataloader, model, loss_fn, 3, log)
print("Done!")

Epoch 50
-------------------------------
loss: 70.629166  [   10/ 6841]
loss: 99.193222  [ 1010/ 6841]
loss: 90.789291  [ 2010/ 6841]
loss: 64.358849  [ 3010/ 6841]
loss: 256.345795  [ 4010/ 6841]
loss: 44.826057  [ 5010/ 6841]
loss: 37.815662  [ 6010/ 6841]
Test Error: 
 Accuracy: 28.1%, Avg loss: 81.431225 

Epoch 100
-------------------------------
loss: 192.329636  [   10/ 6841]
loss: 27.040537  [ 1010/ 6841]
loss: 181.426376  [ 2010/ 6841]
loss: 96.111893  [ 3010/ 6841]
loss: 45.298332  [ 4010/ 6841]
loss: 49.139008  [ 5010/ 6841]
loss: 5.637322  [ 6010/ 6841]
Test Error: 
 Accuracy: 37.6%, Avg loss: 70.368309 

Epoch 150
-------------------------------
loss: 34.033173  [   10/ 6841]
loss: 502.091461  [ 1010/ 6841]
loss: 100.951790  [ 2010/ 6841]
loss: 28.320698  [ 3010/ 6841]
loss: 115.768860  [ 4010/ 6841]
loss: 39.956039  [ 5010/ 6841]
loss: 41.150547  [ 6010/ 6841]
Test Error: 
 Accuracy: 30.0%, Avg loss: 74.545259 

Epoch 200
-------------------------------
loss: 53.244251  [

In [18]:
torch.save(model.state_dict(), "model 51 20 v2 perc12")

In [306]:
torch.load("model 51 20")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [6]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model 51 20"))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [14]:
epochs=10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    test_loop(test_dataloader, model, loss_fn, 3, True)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2344.025424 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.977581 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.015881 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.980488 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.068672 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.973897 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.983285 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.107387 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.960891 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.968734 

